In [ ]:
!pip3 install tqdm -qq
!pip3 install tokenizers==0.13.0 -qq

In [ ]:
import json
import re
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
commentary = 'commentary.jsonl'

In [ ]:
with open(commentary) as file:
    entries = json.load(file)

In [ ]:
with open('bible-books-lookup.json') as file:
    book_info = json.load(file)

# Convert Verse References to Lists

In [ ]:
def convert_vref(ranges):
    for vref in ranges:
        # get the key to the value we want to change later
        key = next(iter(vref))
        references = list(vref.values())
        refs = []
        
        for reference in references:
            pattern = r'(?:(\w+(?:\s\w+)?)\s)?(\d+):(\d+)(?:-(\d+))?'

            # Use the re.match() function to find the matches
            match = re.match(pattern, reference)

            # Extract the book, chapter, and verse from the match
            book = match.group(1)
            chapter = match.group(2)
            verse_start = match.group(3)
            verse_end = match.group(4)
            
            for i in book_info:
                if i['name'] == book:
                    abbr = i['abbr']

            # Determine if it's a single verse or a verse range
            if verse_end:
                verse_refs = []
                verse_range = list(range(int(verse_start), int(verse_end) + 1))
                for i in verse_range:
                    verse_refs.append(f'{abbr} {chapter}:{i}')
            else:
                verse_refs = [f'{abbr} {chapter}:{verse_start}']

            refs.append(verse_refs)
        
        vref[key] = refs

# Translate "Text"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-3.3B")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-3.3B")

In [ ]:
def translate_texts(lang_code):
    translations = []
    for i in entries:
        for j in i['text']:
            inputs = tokenizer(j, return_tensors="pt").to(model.device)
            translated_tokens = model.generate(
                        **inputs, 
                        forced_bos_token_id=tokenizer.lang_code_to_id[lang_code], 
                        max_length=30
                    )
            target_translation = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
            translations.append(target_translation)
            
    return translations

In [ ]:
for entry in entries:
    convert_vref(entry['vrefRanges'])
    entry['spa_text'] = translate_texts('spa_Latn')
    entry['por_text'] = translate_texts('por_Latn')
    entry['fra_text'] = translate_texts('fra_Latn')
    entry['ita_text'] = translate_texts('ita_Latn')
    entry['arb_text'] = translate_texts('arb_Arab')
    print(entry)

In [ ]:
entries